In [37]:
import pandas as pd
import numpy as np
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
import pickle
from keras.preprocessing import sequence,text

In [31]:
df = pd.read_csv("data/train.tsv", sep='\t')
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [32]:
df["Sentiment"].value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [33]:
print("Mean of word count in test set",np.mean(df['Phrase'].apply(lambda x: len(x.split()))))
print("Max of word count in test set",np.max(df['Phrase'].apply(lambda x: len(x.split()))))
print("Min of word count in test set",np.min(df['Phrase'].apply(lambda x: len(x.split()))))

Mean of word count in test set 7.203364090734333
Max of word count in test set 52
Min of word count in test set 0


In [34]:
results = set()
df['Phrase'].str.lower().str.split().apply(results.update)
print("Unique  words in phrases", len(results))

Unique  words in phrases 16531


In [35]:
x = df["Phrase"]
y = df["Sentiment"]

# trainx, testx, trainy, testy = train_test_split(x,y, test_size=0.33, random_state=42)

In [38]:
max_features = 17000
max_words = 55

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x)
x_train = tokenizer.texts_to_sequences(x)
# X_val = tokenizer.texts_to_sequences(X_val_text)
# X_test = tokenizer.texts_to_sequences(test_text)

x1 = sequence.pad_sequences(x_train, maxlen=max_words)
# X_val = sequence.pad_sequences(X_val, maxlen=max_words)
# X_test = sequence.pad_sequences(X_test, maxlen=max_words)

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # loading
# with open('tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)
target = to_categorical(y)

In [39]:
model=Sequential()
model.add(Embedding(max_features,250,mask_zero=True))
model.add(LSTM(128,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 250)         4250000   
_________________________________________________________________
lstm_5 (LSTM)                (None, None, 128)         194048    
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 325       
Total params: 4,493,781
Trainable params: 4,493,781
Non-trainable params: 0
_________________________________________________________________


In [40]:
%%time
history=model.fit(x1, target, epochs=5, batch_size=50, verbose=1)

Epoch 1/5
156060/156060 [==============================] - 1326s 8ms/step - loss: 0.9540 - acc: 0.6154
Epoch 2/5
156060/156060 [==============================] - 1321s 8ms/step - loss: 0.7706 - acc: 0.6837
Epoch 3/5
156060/156060 [==============================] - 1331s 9ms/step - loss: 0.7005 - acc: 0.7089
Epoch 4/5
156060/156060 [==============================] - 1852s 12ms/step - loss: 0.6523 - acc: 0.7277
Epoch 5/5
156060/156060 [==============================] - 2216s 14ms/step - loss: 0.6167 - acc: 0.7419
CPU times: user 5h 57min 56s, sys: 22min 23s, total: 6h 20min 19s
Wall time: 2h 14min 9s


In [41]:
from keras.models import load_model

model.save('sentiment_model.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('sentiment_model.h5')

In [28]:
df1 = pd.read_csv("data/test.tsv", sep='\t')
df1.head()


something
CPU times: user 565 µs, sys: 48 µs, total: 613 µs
Wall time: 148 µs
